In [308]:
%config InlineBackend.figure_formats = ['svg']

In [309]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [310]:
class Layer(object):

    def forward(self, x):
        pass

    def backward(self, dy):
        pass

    def update(self, lr=0.1):
        pass

In [321]:
class LinearLayer(Layer):

    def __init__(self, input, output) -> None:
        self.w = np.random.normal(
            loc=0.0,
            scale=pow(input, -0.5),
            size=(input, output)
        )
        self.dw = self.w * 0.1

    def forward(self, x):
        self.x = x
        y = np.dot(x, self.w)
        return y

    def backward(self, dy):
        dx = np.dot(dy, self.w.T)
        if dy.ndim == 1:
            self.dw = np.dot(self.x.reshape(-1, 1), dy.reshape(1, -1))
        else:
            self.dw = np.dot(self.x.T, dy)

        return dx

    def update(self, lr=0.1):
        self.w += lr * self.dw

In [312]:
x = np.array([
    [0.01, 0.01],
    [0.01, 0.99],
    [0.99, 0.01],
    [0.99, 0.99],
])
t = np.array([
    [0.99, 0.01],
    [0.99, 0.01],
    [0.99, 0.01],
    [0.01, 0.99],
])

In [313]:
layer = LinearLayer(2, 2)

In [314]:
y = layer.forward(x)
print("y: ", y)
print("t: ", t)

loss = t - y
print("loss: ", loss)

layer.backward(loss)
# print("dy: ", dy)
print('dw:', layer.dw)

layer.w -= layer.dw
y = layer.forward(x)
print("y: ", y)

y:  [[ 0.00657999 -0.0122947 ]
 [ 0.37081061 -0.95137739]
 [ 0.28718847 -0.27809218]
 [ 0.6514191  -1.21717488]]
t:  [[0.99 0.01]
 [0.99 0.01]
 [0.99 0.01]
 [0.01 0.99]]
loss:  [[ 0.98342001  0.0222947 ]
 [ 0.61918939  0.96137739]
 [ 0.70281153  0.28809218]
 [-0.6414191   2.20717488]]
dw: [[ 0.02446029  0.78986443]
 [-0.00163858  1.        ]]
y:  [[ 0.00635177 -0.03019334]
 [ 0.37218821 -1.94927604]
 [ 0.26298917 -1.07005797]
 [ 0.6288256  -2.98914067]]


In [322]:
class SigmoidLayer(Layer):

    def forward(self, x):
        # scipy.special.expit(x)
        self.y = 1 / (1 + np.exp(-x))
        return self.y

    def backward(self, dy):
        dx = dy * (1.0 - self.y) * self.y
        return dx

In [331]:
class NeuralNetwork(object):

    def __init__(self, input, hidden, output) -> None:
        self.layers = [
            LinearLayer(input, hidden),
            SigmoidLayer(),
            LinearLayer(hidden, output),
            SigmoidLayer(),
        ]

    def forward(self, x: np.ndarray):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, dy):
        for layer in self.layers[::-1]:
            dy = layer.backward(dy)
        return dy

    def update(self, lr=0.1):
        for layer in self.layers[::-1]:
            layer.update(lr)

In [341]:
net = NeuralNetwork(2, 30, 2)

x_batch = np.array([
    [0.01, 0.01],
    [0.01, 0.99],
    [0.99, 0.01],
    [0.99, 0.99],
])
t_batch = np.array([
    [0.99, 0.01],
    [0.01, 0.99],
    [0.01, 0.99],
    [0.99, 0.01],
])

y = net.forward(x_batch)
print('x:', x_batch)
print('t:', t_batch)
print("y:", y)

epoch = 1000000
bar = tqdm(range(epoch))
for i in bar:
    # for i in range(4):
    y = net.forward(x_batch)
    dy = t_batch - y

    if i % 1000 == 0:
        loss = np.sum(dy ** 2)
        bar.set_postfix(dict(loss=f"{loss:0.3}"))

    dx = net.backward(dy)
    net.update()


y = net.forward(x_batch)
print('x:', x_batch)
print('t:', t_batch)
print("y:", y)


x: [[0.01 0.01]
 [0.01 0.99]
 [0.99 0.01]
 [0.99 0.99]]
t: [[0.99 0.01]
 [0.01 0.99]
 [0.01 0.99]
 [0.99 0.01]]
y: [[0.4606262  0.36124101]
 [0.46642679 0.39675836]
 [0.455298   0.35801365]
 [0.4669981  0.38802752]]


100%|██████████| 1000000/1000000 [00:39<00:00, 25560.46it/s, loss=1.79e-08]

x: [[0.01 0.01]
 [0.01 0.99]
 [0.99 0.01]
 [0.99 0.99]]
t: [[0.99 0.01]
 [0.01 0.99]
 [0.01 0.99]
 [0.99 0.01]]
y: [[0.98994353 0.01005789]
 [0.0100433  0.98995651]
 [0.01004635 0.98995348]
 [0.9899602  0.01003887]]
